In [1]:
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
from sklearn.ensemble import RandomForestClassifier
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from time import time

SKIP_TUNING=True

# Dataset loading
dataset = pd.read_csv(r"./NKI_bin_event.csv", sep=',')

dataset = dataset.drop("Patient", axis=1)
dataset = dataset.drop("survival", axis=1)
dataset = dataset.drop("surv_binary_event", axis=1)
dataset = dataset.drop("survival_binary", axis=1)
dataset = dataset.drop("survival_quartiles", axis=1)
dataset = dataset.drop("eventdeath", axis=1)
dataset = dataset.drop("ID", axis=1)
dataset = dataset.drop("barcode", axis=1)
dataset = dataset.drop("timerecurrence", axis=1)
dataset = dataset.drop("Unnamed: 0", axis=1)

TARGET_COLUMN = "surv_quartile_event"

train_idx = np.random.rand(len(dataset)) < 0.8
train_set = dataset[train_idx]
test_set  = dataset[~train_idx]

train_y = train_set.loc[:, TARGET_COLUMN]
train_x = train_set.drop(TARGET_COLUMN, axis=1)
min_max_scaler = preprocessing.MinMaxScaler()
train_x= min_max_scaler.fit_transform(train_x)
train_y = np.reshape(train_y.to_numpy(), (-1, 1))
train_y= min_max_scaler.fit_transform(train_y)

test_y = test_set.loc[:, TARGET_COLUMN]
test_x = test_set.drop(TARGET_COLUMN, axis=1)
test_x= min_max_scaler.fit_transform(test_x)
test_y = np.reshape(test_y.to_numpy(), (-1, 1))
test_y= min_max_scaler.fit_transform(test_y)


In [1]:
#Load libraries
import tensorflow
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [2]:
plt.style.use('fivethirtyeight')

In [14]:
#Show the number of missing (NAN, NaN, na) data for each column
dataset.isnull().sum()

age                0
survival_binary    0
chemo              0
hormonal           0
amputation         0
histtype           0
diam               0
posnodes           0
grade              0
angioinv           0
lymphinfil         0
esr1               0
G3PDH_570          0
Contig45645_RC     0
Contig44916_RC     0
D25272             0
J00129             0
Contig29982_RC     0
Contig56678_RC     0
Contig53047_RC     0
Contig19551        0
Contig47230_RC     0
Contig46501_RC     0
Contig20749_RC     0
AL157500           0
AL157502           0
Contig37376_RC     0
Contig45395_RC     0
X98307             0
AL157505           0
                  ..
NM_002274          0
NM_001546          0
NM_002275          0
NM_002276          0
NM_001548          0
Contig51486_RC     0
NM_000824          0
NM_001554          0
NM_000826          0
NM_001555          0
NM_018942          0
NM_001565          0
NM_002299          0
NM_018950          0
NM_018952          0
Contig55606_RC     0
D25217       

In [18]:
dataset.shape

(272, 1565)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.layers import Dense, Dropout, Input
from keras.models import Model,Sequential
from keras.datasets import mnist
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam, SGD

Using TensorFlow backend.


In [3]:
def adam_optimizer():
    return Adam(lr=0.0002, beta_1=0.5)

In [4]:
def create_generator():
    generator=Sequential()
    generator.add(Dense(units=256,input_dim=1565))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=512))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=1024))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=1565, activation='tanh'))
    
    generator.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
    return generator
g=create_generator()
g.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               400896    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              525312    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1565)             

In [5]:
def create_discriminator():
    discriminator=Sequential()
    discriminator.add(Dense(units=1024,input_dim=1565))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
       
    
    discriminator.add(Dense(units=512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
       
    discriminator.add(Dense(units=256))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Dense(units=1, activation='sigmoid'))
    
    discriminator.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
    return discriminator
d =create_discriminator()
d.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1024)              1603584   
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)              

In [6]:
def create_gan(discriminator, generator):
    discriminator.trainable=False
    gan_input = Input(shape=(1565,))
    x = generator(gan_input)
    gan_output= discriminator(x)
    gan= Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
    return gan
gan = create_gan(d,g)
gan.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1565)              0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 1565)              2661917   
_________________________________________________________________
sequential_2 (Sequential)    (None, 1)                 2259969   
Total params: 4,921,886
Trainable params: 2,661,917
Non-trainable params: 2,259,969
_________________________________________________________________


In [7]:
def training(epochs=1, batch_size=128):
    
    #Loading the data
    batch_count = train_set.shape[0] / batch_size
    half_batch = int(batch_size / 2)
    # Creating GAN
    generator= create_generator()
    discriminator= create_discriminator()
    gan = create_gan(discriminator, generator)
    
    txn = train_x
    tyn = train_set[TARGET_COLUMN].to_numpy()
    tyn = np.reshape(tyn, (-1, 1))
    ts = np.concatenate((txn, tyn), axis=1)

    for e in range(1,epochs+1 ):
        print("Epoch %d" %e)
        for _ in tqdm(range(batch_size)):
        #generate  random noise as an input  to  initialize the  generator
            noise= np.random.normal(0,1, [batch_size, 1565])
            
            # Generate fake data from noised input
            generated_data = generator.predict(noise)
            
            # Get a random set of  real data
            image_batch = ts[np.random.randint(low=0,high=ts.shape[0],size=batch_size)]
                  # Train the discriminator

            #Construct different batches of  real and fake data 
            X= np.concatenate([image_batch, generated_data])
            
            # Labels for generated and real data
            y_dis=np.zeros(2*batch_size)
            y_dis[:batch_size]=1
            
            #Pre train discriminator on  fake and real data  before starting the gan. 
            discriminator.trainable=True
            discriminator.train_on_batch(X, y_dis)
            
            #Tricking the noised input of the Generator as real data
            noise= np.random.normal(0,1, [batch_size, 1565])
            
            # Plot the progress
            y_gen = np.ones(batch_size)
            
            # During the training of gan, 
            # the weights of discriminator should be fixed. 
            #We can enforce that by setting the trainable flag
            discriminator.trainable=False
            
            #training  the GAN by alternating the training of the Discriminator 
            #and training the chained GAN model with Discriminator’s weights freezed.
            gan.train_on_batch(noise, y_gen)
    
    return generator

In [8]:
generator=training(epochs=2048, batch_size=128)

  0%|                                                                                          | 0/128 [00:00<?, ?it/s]

Epoch 1


  2%|█▉                                                                                | 3/128 [00:00<00:04, 25.86it/s]

Epoch 2


  2%|█▉                                                                                | 3/128 [00:00<00:05, 24.58it/s]

...

  2%|█▉                                                                                | 3/128 [00:00<00:04, 26.31it/s]

Epoch 2047


  2%|█▉                                                                                | 3/128 [00:00<00:04, 26.31it/s]

Epoch 2048


100%|████████████████████████████████████████████████████████████████████████████████| 128/128 [00:04<00:00, 26.01it/s]


In [9]:
# Re-train NN using generated data
nFake = 4096

model = Sequential([
    Dense(12, activation='relu', input_shape=( 1564 ,)),
    Dense(15, activation='relu'),
    Dense(1, activation='sigmoid')
])

opt = SGD(lr=0.0001, momentum=0.9, clipnorm=1.0)

model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

noise= np.random.normal(0,1, [nFake, 1565])
            
# Generate fake data from noised input
generated_data = generator.predict(noise)

g_x=generated_data[:, :-1]
g_y=generated_data[:, -1]

Y = np.concatenate((train_y, g_y.reshape(-1, 1)))
X = np.concatenate((train_x, g_x))

print(g_x[128:200])
print(g_y[128:200])

hist = model.fit(X, Y,
          batch_size=128, epochs=256,  validation_split=0.2)



[[ 0.30463964 -0.00873254  0.00910961 ...  0.5970179   0.30433303
   0.86365104]
 [ 0.4963358   0.9864344  -0.00278238 ...  0.2481802   0.42883173
   0.53887486]
 [ 0.6491388  -0.09956334 -0.04126145 ...  0.54266125  0.61954916
   0.80892915]
 ...
 [ 0.6717865  -0.04599546 -0.03980567 ...  0.14412221  0.47460923
   0.49209374]
 [ 0.354806    0.98767185  0.00179723 ...  0.25418702  0.25075036
   0.46482515]
 [ 0.6657182   0.99733514  0.00775185 ...  0.61366415  0.39385346
   0.5891894 ]]
[ 0.9999432   0.9918008   0.98891306  0.99393725  0.707639    0.99803054
  0.9436842   0.9923641   0.99362725  0.9730866   0.9323038   0.9993948
  0.68848157  0.9930795   0.99645054  0.9998378   0.9858101   0.98529613
  0.9884795   0.9977765   0.9986551   0.9950376   0.99677914  0.9998941
  0.9991422   0.99918604  0.5495876   0.9959067  -0.5340313   0.99991804
  0.9984699   0.91031265  0.92546046  0.99438936  0.9619213   0.9963959
  0.97333044  0.9989501   0.94708216  0.99905974  0.9942336   0.99577826


Epoch 46/256
3449/3449 [==============================] - 0s 44us/step - loss: 0.2526 - accuracy: 0.0159 - val_loss: 0.1841 - val_accuracy: 0.0000e+00
Epoch 47/256
3449/3449 [==============================] - 0s 43us/step - loss: 0.2526 - accuracy: 0.0159 - val_loss: 0.1841 - val_accuracy: 0.0000e+00
Epoch 48/256
3449/3449 [==============================] - 0s 44us/step - loss: 0.2525 - accuracy: 0.0159 - val_loss: 0.1841 - val_accuracy: 0.0000e+00
Epoch 49/256
3449/3449 [==============================] - 0s 44us/step - loss: 0.2525 - accuracy: 0.0159 - val_loss: 0.1841 - val_accuracy: 0.0000e+00
Epoch 50/256
3449/3449 [==============================] - 0s 44us/step - loss: 0.2525 - accuracy: 0.0159 - val_loss: 0.1840 - val_accuracy: 0.0000e+00
Epoch 51/256
3449/3449 [==============================] - 0s 43us/step - loss: 0.2524 - accuracy: 0.0159 - val_loss: 0.1841 - val_accuracy: 0.0000e+00
Epoch 52/256
3449/3449 [==============================] - 0s 44us/step - loss: 0.2524 - accura

Epoch 154/256
3449/3449 [==============================] - 0s 42us/step - loss: 0.2490 - accuracy: 0.0159 - val_loss: 0.1824 - val_accuracy: 0.0000e+00
Epoch 155/256
3449/3449 [==============================] - 0s 44us/step - loss: 0.2489 - accuracy: 0.0159 - val_loss: 0.1823 - val_accuracy: 0.0000e+00
Epoch 156/256
3449/3449 [==============================] - 0s 42us/step - loss: 0.2489 - accuracy: 0.0159 - val_loss: 0.1824 - val_accuracy: 0.0000e+00
Epoch 157/256
3449/3449 [==============================] - 0s 43us/step - loss: 0.2489 - accuracy: 0.0159 - val_loss: 0.1824 - val_accuracy: 0.0000e+00
Epoch 158/256
3449/3449 [==============================] - 0s 43us/step - loss: 0.2489 - accuracy: 0.0159 - val_loss: 0.1824 - val_accuracy: 0.0000e+00
Epoch 159/256
3449/3449 [==============================] - 0s 43us/step - loss: 0.2488 - accuracy: 0.0159 - val_loss: 0.1823 - val_accuracy: 0.0000e+00
Epoch 160/256
3449/3449 [==============================] - 0s 43us/step - loss: 0.2488 -

Epoch 208/256
3449/3449 [==============================] - 0s 43us/step - loss: 0.2474 - accuracy: 0.0159 - val_loss: 0.1816 - val_accuracy: 0.0000e+00
Epoch 209/256
3449/3449 [==============================] - 0s 44us/step - loss: 0.2474 - accuracy: 0.0159 - val_loss: 0.1815 - val_accuracy: 0.0000e+00
Epoch 210/256
3449/3449 [==============================] - 0s 43us/step - loss: 0.2474 - accuracy: 0.0159 - val_loss: 0.1815 - val_accuracy: 0.0000e+00
Epoch 211/256
3449/3449 [==============================] - 0s 44us/step - loss: 0.2473 - accuracy: 0.0159 - val_loss: 0.1815 - val_accuracy: 0.0000e+00
Epoch 212/256
3449/3449 [==============================] - 0s 43us/step - loss: 0.2473 - accuracy: 0.0159 - val_loss: 0.1815 - val_accuracy: 0.0000e+00
Epoch 213/256
3449/3449 [==============================] - 0s 42us/step - loss: 0.2473 - accuracy: 0.0159 - val_loss: 0.1816 - val_accuracy: 0.0000e+00
Epoch 214/256
3449/3449 [==============================] - 0s 45us/step - loss: 0.2473 -

In [10]:
model.evaluate(test_x, test_y)[1]

56/56 [==============================] - 0s 89us/step


0.3214285671710968

In [1]:
## Analysis
#The model accurately identified people surviving breastcancer or not with 32.1% accuracy on the test data !